In [ ]:
import aliases

In [ ]:
from everest.utilities import classtools

In [ ]:
@classtools.MROClassable
class A:
    @classtools.MROClass
    class AA:
        @classmethod
        def foo(self):
            return 'foo'

class B(A):
    class AA:
        @classmethod
        def foo(self):
            return super().foo() * 2

class C(A):
    ...

class D(B, C):
    class AA:
        @classmethod
        def foo(self):
            return super().foo() + '_extra'

In [ ]:
D.AA.foo()

In [ ]:
from everest import incision

In [ ]:
from everest.utilities import classtools

In [ ]:
class MyClass:
    def __init_subclass__(cls, **kwargs):
        cls.__getitem__ = lambda cls, arg: arg
        super().__init_subclass__(**kwargs)
    def __getitem__(cls, arg):
        raise Exception

In [ ]:
MyClass.__dict__.items()

In [ ]:
dir(MyClass)

In [ ]:
class MySubClass(MyClass):
    ...

In [ ]:
MySubClass()[0]

In [ ]:
import typing

In [ ]:
typing.NoReturn

In [ ]:
Foo[0]

In [ ]:
class Metric(incision.Incisable):
    ...


class Ordered(Metric):
    ...

In [ ]:
Metric.Incisor

In [ ]:
isinstance(1, Metric.Incisor)

In [ ]:
Metric.incmeths

In [ ]:
import collections.abc as _collabc

In [ ]:
from __future__ import annotations

In [ ]:
def myfn() -> int | str:
    return 1

In [ ]:
annotations

In [ ]:
    def incision_methods(cls) -> \
            _collabc.Iterator[tuple[type, _collabc.Callable]] \
            :
        ...

In [ ]:
from collections.abc import Iterator

In [ ]:
from typing import Callable

In [ ]:
import typing

In [ ]:
typing.Iterator

In [ ]:
Callable

In [ ]:
Iterator[int]

In [ ]:
isinstance(Metric, callable)

In [ ]:
class MyClass:
    name: str = None

In [ ]:
MyClass

In [ ]:
foo = 1
class MyClass:
    def myfn(self):
        _foo = global foo
        print(foo)

In [ ]:
MyClass.__name__

In [ ]:
type(

In [ ]:
typing.get_type_hints(MyClass)